In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
%%time
bdinfo_df = pd.read_csv('./국토교통부_건축물대장_표제부/bdinfo_baseline_maindata_inter_checkpoint_4.csv')
print(bdinfo_df.shape)
bdinfo_df.head()

<string>:2: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.


(935428, 51)
Wall time: 7.23 s


,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,주_부속_구분,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,사용승인년도
0,11440-21639,집합,표제부,0,주건축물,177.62,105.300,59.28,393.130,385.210,216.87,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트 평스라브,12.00,4,1,0,0,0,0.0,393.130,0,0.0,0,0.0,0,0.00,4,46.0,20040323.0,NaN,0.0,0,NaN,0,NaN,0,20101230,201412,서울특별시 마포구 서교동 484-16,4,일반,20040323.0,20040323.0,20101230,20170209,2004
1,11590-100182663,집합,표제부,1,주건축물,389.48,232.420,59.67,947.740,659.890,169.43,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,11.26,4,1,0,0,0,0.0,947.740,0,0.0,0,0.0,8,287.85,2,11.5,20080826.0,NaN,0.0,0,NaN,0,NaN,0,20110119,201412,서울특별시 동작구 상도동 302-28,10,일반,20080826.0,20080826.0,20110119,20190530,2008
2,11320-100182241,집합,표제부,0,주건축물,208.00,81.085,38.98,268.165,268.165,128.93,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,10.40,4,0,0,0,0,0.0,268.165,0,0.0,0,0.0,3,0.00,1,0.0,20080721.0,NaN,0.0,0,NaN,0,NaN,0,20110202,201412,서울특별시 도봉구 쌍문동 380-24,4,일반,20080721.0,20080721.0,20110202,20190530,2008
3,11440-32174,집합,표제부,0,주건축물,314.00,180.380,57.45,627.880,627.880,199.96,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,평스라브,13.30,5,0,0,0,0,0.0,627.880,0,0.0,0,0.0,7,80.50,1,11.5,20050901.0,NaN,0.0,0,NaN,0,NaN,0,20110125,201412,서울특별시 마포구 성산동 200-260,8,일반,20050901.0,20050901.0,20110125,20171205,2005
4,11740-100179671,집합,표제부,0,주건축물,0.00,564.010,0.00,6401.190,6401.190,0.00,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트,45.60,14,0,3,0,0,0.0,6401.190,0,0.0,0,0.0,0,0.00,0,0.0,20090311.0,NaN,0.0,0,NaN,0,NaN,0,20110122,201412,서울특별시 강동구 강일동 674-0,78,일반,20090311.0,20090311.0,20100918,20200428,2009


In [3]:
area_df = bdinfo_df[['지번주소', '기준년월', '연면적(㎡)']]
area_df['연면적'] = area_df.groupby(['지번주소', '기준년월']).transform('sum')['연면적(㎡)']
area_df = area_df.drop(columns=['연면적(㎡)']).drop_duplicates().reset_index(drop=True)
area_df.shape

(716540, 3)

In [4]:
%%time
area_df.to_csv('./국토교통부_건축물대장_표제부/표제부상_연면적.csv', index=False)

Wall time: 3.08 s


In [6]:
living_area_df = bdinfo_df[['지번주소', '기준년월', '용적_률_산정_연면적(㎡)']]
living_area_df['용적률산정연면적'] = living_area_df.groupby(['지번주소', '기준년월']).transform('sum')['용적_률_산정_연면적(㎡)']
living_area_df = living_area_df[['지번주소', '기준년월', '용적률산정연면적']].drop_duplicates().reset_index(drop=True)
living_area_df.shape

(716540, 3)

In [7]:
%%time
living_area_df.to_csv('./국토교통부_건축물대장_표제부/표제부상_용적률산정연면적.csv', index=False)

Wall time: 3.18 s
